In [1]:
import numpy as np
import pandas as pd

import geopandas as gpd
#from shapely.geometry import Point
#from shapely import wkt
#import geographiclib
#from geographiclib.geodesic import Geodesic
#import utm
#from shapely.ops import nearest_points

import re
import sys
import os
import requests
from bs4 import BeautifulSoup
from lxml import html
import openpyxl
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#import urbanpy as up
#import contextily as ctx

import io
import json

import plotly.graph_objects as go
import plotly.express as px
import plotly as py

pd.set_option('display.max_rows',999)
pd.set_option('display.min_rows',4)
pd.set_option('display.float_format', '{:,.2f}'.format)

warnings.filterwarnings("ignore")

### 20 colores

In [2]:
cl = [e for e in sns.color_palette('rocket',20)]
cl2 = [(int(t[0]*255), int(t[1]*255) , int(t[2]*255)) for t in cl]
colors = ['#%02x%02x%02x' % e for e in cl2]

### idx

In [3]:
idx = pd.IndexSlice

### Bases

In [4]:
dbox = r'c:/Users/Franco/Dropbox/files/clean/'
files = os.listdir(dbox)

In [5]:
sectores = pd.read_excel(dbox+'sectores.xlsx',index_col=[0,1])
laboral = pd.read_excel(dbox+'laboral_final(ajustes).xlsx',index_col=[0,1])
acceso = pd.read_excel(dbox+'accesom_2.xlsx',index_col=[0,1])
accesom = pd.read_excel(dbox+'horas_msII.xlsx',index_col=[0,1]) 
cw = pd.read_excel(dbox+'cwpops.xlsx',index_col=[0,1])
latlongs = pd.read_excel(dbox+'latlongs.xlsx',index_col=[0,1])
msa = pd.read_excel(dbox+'metro_areas.xlsx',index_col=[0,1])

In [8]:
prope_com = pd.read_excel(dbox+'prope_comercial.xlsx',index_col=[0,1])
prope_vi = pd.read_csv(dbox+'prope_vivienda.csv',index_col=[0,1])

In [9]:
es = pd.read_excel(dbox+'es.xlsx',index_col=[0,1])

In [11]:
hom = pd.read_excel(dbox+'homicidios.xlsx',index_col = [0,1])

### Ciudades de referencia

In [26]:
ref = ['Gran Cordoba','Gran Tucuman-Tafi Viejo','Gran Rosario','Gran Mendoza','Mar del Plata-Batan','Bahia Blanca-Cerri','Vitoria','Cali','San Luis Potosi']
ref_prom = ref + ['Promedio Argentina','Promedio Latam']

In [30]:
ref = ['Manizales','Gran Mendoza']

## Funciones

In [28]:
#Tabla 1
def tabla1(ref1=ref):
    vars_laboral = ['Salario Horario (PPP 2011)','Tasa de Desempleo Total','% población con secundario completo Total']
    vars_acceso = ['cutoff_0','cutoff_9']
    vars_sectores = ['Ratio Agro','Ratio Baja Tec','Ratio Resto Ind']
    
    comp = cw.loc[idx[:,ref1],'pops'].to_frame().join(laboral.loc[:,vars_laboral]).join(acceso.loc[:,vars_acceso]).join(sectores.loc[:,vars_sectores])
    comp = comp.rename(columns={'pops':'Población estimada (millones)','cutoff_0':'Masa Salarial como proporción del país','cutoff_9':'Proporción de la masa salarial del país a la que se accede en < 9 hs'})
    
    return BeautifulSoup(comp.sort_values(['Country','Salario Horario (PPP 2011)']).to_html(), 'html.parser')

In [31]:
tabla1()

<table border="1" class="dataframe">
<thead>
<tr style="text-align: right;">
<th></th>
<th></th>
<th>Población estimada (millones)</th>
<th>Salario Horario (PPP 2011)</th>
<th>Tasa de Desempleo Total</th>
<th>% población con secundario completo Total</th>
<th>Masa Salarial como proporción del país</th>
<th>Proporción de la masa salarial del país a la que se accede en &lt; 9 hs</th>
<th>Ratio Agro</th>
<th>Ratio Baja Tec</th>
<th>Ratio Resto Ind</th>
</tr>
<tr>
<th>Country</th>
<th>City</th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<th rowspan="6" valign="top">Argentina</th>
<th>Gran Tucuman-Tafi Viejo</th>
<td>1.45</td>
<td>4.45</td>
<td>10.32</td>
<td>42.95</td>
<td>0.04</td>
<td>0.13</td>
<td>0.96</td>
<td>0.89</td>
<td>0.97</td>
</tr>
<tr>
<th>Gran Cordoba</th>
<td>1.43</td>
<td>4.64</td>
<td>12.75</td>
<td>46.18</td>
<td>0.04</td>
<td>0.87</td>
<td>0.31</td>
<td>1.09</td>
<td>1.51</td>
</tr>
<tr>
<th>Gran 